# Detección de Intrusiones y Análisis de Anomalías en Tráfico de Red mediante Técnicas Estadísticas

**Universidad de La Habana, MATCOM**  
**Curso:** Estadística 2025-2026  
**Proyecto Final:** Análisis Estadístico Aplicado a Seguridad Informática

---

## 1. Introducción al Proyecto

### 1.1 Contexto y Motivación

La seguridad informática es uno de los pilares fundamentales en la infraestructura tecnológica moderna. Los Sistemas de Detección de Intrusiones (IDS) tradicionales, basados en firmas conocidas, presentan limitaciones significativas frente a ataques emergentes o modificados (zero-day attacks). 

Este proyecto propone un enfoque complementario basado en **análisis estadístico del comportamiento del tráfico de red**, permitiendo identificar patrones anómalos sin depender exclusivamente de firmas previamente catalogadas. Este tipo de aproximación resulta especialmente relevante en entornos dinámicos donde los ataques evolucionan constantemente.

### 1.2 Objetivos del Análisis

Este estudio busca responder **tres preguntas de investigación fundamentales**:

**Pregunta 1 (Análisis Comparativo):** ¿Existen diferencias estadísticamente significativas en el comportamiento de variables de flujo de red —como `src_bytes`, `dst_bytes` y `duration`— entre el tráfico normal y los distintos tipos de ataques (DoS, Probe, R2L y U2R)?

**Pregunta 2 (Reducción Dimensional):** ¿Es posible reducir la dimensionalidad de las 41 características del tráfico de red mediante Análisis de Componentes Principales (PCA), conservando al menos el 95% de la varianza explicada, y cómo impacta esta reducción en la visualización y separación de los distintos tipos de ataques?

**Pregunta 3 (Clasificación Comparativa):** ¿Qué técnica de clasificación estadística, Regresión Logística o K-Vecinos más Cercanos (K-NN), ofrece una mayor sensibilidad para detectar ataques raros (como U2R) en comparación con ataques volumétricos más comunes (como DoS)?

### 1.3 Dataset: NSL-KDD

**Fuente:** [NSL-KDD en Kaggle](https://www.kaggle.com/datasets/hassan06/nslkdd)

El dataset NSL-KDD es una versión refinada del clásico KDD Cup 1999, diseñada específicamente para eliminar redundancias y sesgos presentes en el conjunto original. Es ampliamente reconocido como estándar académico para la evaluación de algoritmos de detección de intrusiones.

**Características principales:**
- **41 variables predictoras** + 1 variable objetivo (`attack_type`) + 1 nivel de dificultad
- **Tipos de ataques:** Normal, DoS (Denial of Service), Probe (escaneo/sondeo), R2L (Remote to Local), U2R (User to Root)
- **Conjunto de entrenamiento:** 25,192 observaciones
- **Conjunto de prueba:** 22,544 observaciones

**Categorización de variables:**
- **Básicas:** Derivadas de cabeceras TCP/IP (duration, protocol_type, src_bytes, dst_bytes, flag)
- **De contenido:** Información sobre el payload (num_failed_logins, root_shell, etc.)
- **De tráfico:** Estadísticas temporales orientadas a detectar patrones (count, serror_rate, etc.)

---

## 1.4 Carga de Datos y Preparación Inicial

### Imports y Configuración

In [ ]:
# Importación de librerías necesarias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

# Configuración de visualizaciones
warnings.filterwarnings('ignore')
%matplotlib inline

# Estilo global para mantener consistencia en todas las visualizaciones
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['font.size'] = 11
plt.rcParams['axes.labelsize'] = 11
plt.rcParams['axes.titlesize'] = 13
plt.rcParams['xtick.labelsize'] = 10
plt.rcParams['ytick.labelsize'] = 10
plt.rcParams['legend.fontsize'] = 10

# Paleta de colores consistente para categorías de ataque
# Se utilizará en todas las visualizaciones del proyecto
attack_colors = {
    'Normal': '#2ecc71',    # Verde - Tráfico legítimo
    'DoS': '#e74c3c',       # Rojo - Ataques de denegación de servicio
    'Probe': '#f39c12',     # Naranja - Ataques de reconocimiento
    'R2L': '#9b59b6',       # Morado - Acceso remoto no autorizado
    'U2R': '#34495e'        # Gris oscuro - Escalada de privilegios
}

print("✅ Librerías importadas y configuración de visualización establecida")

### Carga de Datos

In [ ]:
# Definir los nombres de las columnas (43 columnas en total)
col_names = [
    "duration", "protocol_type", "service", "flag", "src_bytes",
    "dst_bytes", "land", "wrong_fragment", "urgent", "hot",
    "num_failed_logins", "logged_in", "num_compromised", "root_shell",
    "su_attempted", "num_root", "num_file_creations", "num_shells",
    "num_access_files", "num_outbound_cmds", "is_host_login",
    "is_guest_login", "count", "srv_count", "serror_rate",
    "srv_serror_rate", "rerror_rate", "srv_rerror_rate", "same_srv_rate",
    "diff_srv_rate", "srv_diff_host_rate", "dst_host_count",
    "dst_host_srv_count", "dst_host_same_srv_rate",
    "dst_host_diff_srv_rate", "dst_host_same_src_port_rate",
    "dst_host_srv_diff_host_rate", "dst_host_serror_rate",
    "dst_host_srv_serror_rate", "dst_host_rerror_rate",
    "dst_host_srv_rerror_rate", "attack_type", "difficulty_level"
]

# Cargar los datasets
# Nota: Ajusta las rutas según tu estructura de carpetas
train_df = pd.read_csv('Data/KDDTrain+_20Percent.txt', 
                       names=col_names, 
                       header=None)

test_df = pd.read_csv('Data/KDDTest+.txt', 
                      names=col_names, 
                      header=None)

# Crear variable binaria para clasificación binaria (Normal vs. Ataque)
train_df['is_attack'] = (train_df['attack_type'] != 'normal').astype(int)
test_df['is_attack'] = (test_df['attack_type'] != 'normal').astype(int)

# Mostrar información básica
print(f"📊 Datos de entrenamiento: {train_df.shape}")
print(f"📊 Datos de prueba: {test_df.shape}")
print(f"\n✅ Datasets cargados exitosamente")

# Distribución binaria inicial
print(f"\n🎯 Distribución binaria en entrenamiento (Normal vs. Ataque):")
print(train_df['is_attack'].value_counts(normalize=True).round(4))

print(f"\n🎯 Distribución binaria en prueba (Normal vs. Ataque):")
print(test_df['is_attack'].value_counts(normalize=True).round(4))

## 1.5 Mapeo de Categorías de Ataque

El dataset NSL-KDD contiene 39 tipos de ataques específicos que se agrupan en 4 categorías principales más la clase normal. A continuación se realiza el mapeo oficial según la documentación del Canadian Institute for Cybersecurity:

- **Normal:** Tráfico de red legítimo
- **DoS (Denial of Service):** Ataques que buscan denegar el servicio mediante sobrecarga de recursos
- **Probe (Probing/Scanning):** Ataques de reconocimiento que escanean la red en busca de vulnerabilidades
- **R2L (Remote to Local):** Intentos de acceso no autorizado desde una máquina remota
- **U2R (User to Root):** Intentos de escalada de privilegios de usuario normal a superusuario

In [ ]:
# Diccionario oficial de mapeo de ataques específicos a categorías generales
# Fuente: Documentación oficial NSL-KDD (Canadian Institute for Cybersecurity)
attack_category_mapping = {
    # Tráfico Normal
    'normal': 'Normal',
    
    # DoS (Denial of Service) - Ataques de denegación de servicio
    'back': 'DoS', 'land': 'DoS', 'neptune': 'DoS', 'pod': 'DoS',
    'smurf': 'DoS', 'teardrop': 'DoS', 'mailbomb': 'DoS', 'apache2': 'DoS',
    'processtable': 'DoS', 'udpstorm': 'DoS',
    
    # Probe (Probing/Scanning) - Ataques de reconocimiento
    'ipsweep': 'Probe', 'nmap': 'Probe', 'portsweep': 'Probe',
    'satan': 'Probe', 'mscan': 'Probe', 'saint': 'Probe',
    
    # R2L (Remote to Local) - Acceso no autorizado desde máquina remota
    'ftp_write': 'R2L', 'guess_passwd': 'R2L', 'imap': 'R2L',
    'multihop': 'R2L', 'phf': 'R2L', 'spy': 'R2L',
    'warezclient': 'R2L', 'warezmaster': 'R2L', 'sendmail': 'R2L',
    'named': 'R2L', 'snmpgetattack': 'R2L', 'snmpguess': 'R2L',
    'xlock': 'R2L', 'xsnoop': 'R2L', 'worm': 'R2L',
    
    # U2R (User to Root) - Escalada de privilegios
    'buffer_overflow': 'U2R', 'loadmodule': 'U2R', 'perl': 'U2R',
    'rootkit': 'U2R', 'httptunnel': 'U2R', 'ps': 'U2R', 'sqlattack': 'U2R'
}

# Aplicar mapeo a ambos datasets
train_df['attack_category'] = train_df['attack_type'].map(attack_category_mapping)
test_df['attack_category'] = test_df['attack_type'].map(attack_category_mapping)

# Verificar que no hay valores sin mapear
print("🔍 Verificación de mapeo de categorías:")
unmapped_train = train_df['attack_category'].isna().sum()
unmapped_test = test_df['attack_category'].isna().sum()
print(f"   Valores sin mapear en train: {unmapped_train}")
print(f"   Valores sin mapear en test: {unmapped_test}")

if unmapped_test > 0:
    print(f"\n⚠️ ADVERTENCIA: Hay {unmapped_test} ataques en test sin categoría asignada.")
    print("   Esto es esperado en NSL-KDD, que incluye ataques nuevos en el conjunto de prueba.")
    print("   Estos registros se filtrarán en análisis posteriores.")

# Crear orden categórico para visualizaciones consistentes
category_order = ['Normal', 'DoS', 'Probe', 'R2L', 'U2R']
train_df['attack_category'] = pd.Categorical(
    train_df['attack_category'], 
    categories=category_order, 
    ordered=True
)
test_df['attack_category'] = pd.Categorical(
    test_df['attack_category'], 
    categories=category_order, 
    ordered=True
)

# Mostrar distribución de categorías
print("\n📊 Distribución de categorías en ENTRENAMIENTO:")
category_dist_train = train_df['attack_category'].value_counts()
category_pct_train = train_df['attack_category'].value_counts(normalize=True) * 100
category_summary_train = pd.DataFrame({
    'Frecuencia': category_dist_train,
    'Porcentaje': category_pct_train.round(2)
})
print(category_summary_train)

print("\n📊 Distribución de categorías en PRUEBA:")
category_dist_test = test_df[test_df['attack_category'].notna()]['attack_category'].value_counts()
category_pct_test = test_df[test_df['attack_category'].notna()]['attack_category'].value_counts(normalize=True) * 100
category_summary_test = pd.DataFrame({
    'Frecuencia': category_dist_test,
    'Porcentaje': category_pct_test.round(2)
})
print(category_summary_test)

print("\n✅ Mapeo de categorías completado exitosamente")

## 1.6 Preparación de Muestra Estratificada para Visualizaciones

Para optimizar el rendimiento de visualizaciones complejas (como scatterplot matrices y pairplots), crearemos una muestra estratificada de 5,000 observaciones que mantenga las proporciones originales de cada categoría de ataque. 

**Nota importante:** Esta muestra se utilizará **exclusivamente para visualizaciones**. Todos los análisis estadísticos (correlaciones, pruebas de hipótesis, modelos) se realizarán sobre el dataset completo.

In [ ]:
# Configuración del muestreo estratificado
sample_size = 5000
random_state = 42  # Para reproducibilidad

# Crear muestra estratificada manteniendo proporciones de cada categoría
train_sample = train_df.groupby('attack_category', group_keys=False).apply(
    lambda x: x.sample(
        n=min(len(x), int(sample_size * len(x) / len(train_df))),
        random_state=random_state
    )
).reset_index(drop=True)

print(f"📊 Muestra estratificada creada: {len(train_sample):,} observaciones")
print(f"\n✅ Verificación de estratificación:")

# Comparar proporciones originales vs. muestra
comparison = pd.DataFrame({
    'Original (%)': train_df['attack_category'].value_counts(normalize=True).sort_index() * 100,
    'Muestra (%)': train_sample['attack_category'].value_counts(normalize=True).sort_index() * 100
})
comparison['Diferencia (pp)'] = (comparison['Muestra (%)'] - comparison['Original (%)']).abs()
print(comparison.round(2))

print("\n💡 Esta muestra se usará únicamente para visualizaciones pesadas.")
print("   Los análisis estadísticos utilizarán el dataset completo.")

---

# 2. ANÁLISIS EXPLORATORIO DE DATOS (EDA)

El Análisis Exploratorio de Datos es fundamental para comprender la estructura, distribución y relaciones presentes en el dataset antes de aplicar técnicas estadísticas avanzadas. Esta sección cumple tres objetivos principales:

1. **Validar la calidad de los datos** y detectar problemas estructurales
2. **Fundamentar decisiones de preparación** que se implementarán en la sección 3
3. **Generar hipótesis preliminares** que guiarán las pruebas estadísticas posteriores

---

## 2.1 Información General del Dataset

En esta primera sección del EDA realizaremos una exploración estructural del dataset para verificar:
- Dimensiones y tipos de datos
- Presencia de valores nulos o faltantes
- Distribución de variables numéricas vs. categóricas
- Uso de memoria, consideraciones computacionales y duplicados

In [ ]:
print("INFORMACIÓN ESTRUCTURAL DEL DATASET")

# 1. Dimensiones
print(f"\n📦 Dimensiones de los datasets:")
print(f"   Entrenamiento: {train_df.shape[0]:,} observaciones × {train_df.shape[1]} variables")
print(f"   Prueba: {test_df.shape[0]:,} observaciones × {test_df.shape[1]} variables")

# 2. Información de tipos de datos
print(f"\n🔢 Distribución de tipos de datos (Train):")
print(train_df.dtypes.value_counts())

# 3. Duplicados
print(f"\n🔁 REGISTROS DUPLICADOS:")
duplicates = train_df.duplicated().sum()
print(f"Filas duplicadas: {duplicates}")

# 4. Separación de variables numéricas y categóricas
numeric_cols = train_df.select_dtypes(include=[np.number]).columns.tolist()
categorical_cols = train_df.select_dtypes(include=['object', 'category']).columns.tolist()

# Remover variables objetivo y auxiliares de las listas
numeric_cols = [col for col in numeric_cols if col not in ['is_attack', 'difficulty_level']]
categorical_cols = [col for col in categorical_cols if col not in ['attack_type', 'attack_category']]

print(f"\n📊 Clasificación de variables predictoras:")
print(f"   Variables numéricas: {len(numeric_cols)}")
print(f"   Variables categóricas: {len(categorical_cols)}")
print(f"   Total de predictoras: {len(numeric_cols) + len(categorical_cols)}")

print(f"\n   Variables categóricas identificadas:")
for col in categorical_cols:
    n_unique = train_df[col].nunique()
    print(f"      - {col}: {n_unique} categorías únicas")

In [ ]:
# Verificación de valores nulos en variables predictoras originales
print("\n🔍 Verificación de valores faltantes:")

# Excluir attack_category del análisis de nulos (es variable derivada)
original_cols = [col for col in train_df.columns if col not in ['attack_category', 'is_attack']]
missing_train = train_df[original_cols].isnull().sum().sum()
missing_test = test_df[original_cols].isnull().sum().sum()

print(f"   Dataset de entrenamiento: {missing_train} valores nulos")
print(f"   Dataset de prueba: {missing_test} valores nulos")

if missing_train == 0 and missing_test == 0:
    print(f"\n   ✅ Excelente: No hay valores faltantes en las variables originales.")
    print(f"      No se requiere imputación de datos.")
else:
    print(f"\n   ⚠️ Se detectaron valores faltantes. Análisis detallado:")
    if missing_train > 0:
        print(f"\n   Columnas con valores nulos en TRAIN:")
        missing_cols_train = train_df[original_cols].isnull().sum()[train_df[original_cols].isnull().sum() > 0]
        print(missing_cols_train)
    if missing_test > 0:
        print(f"\n   Columnas con valores nulos en TEST:")
        missing_cols_test = test_df[original_cols].isnull().sum()[test_df[original_cols].isnull().sum() > 0]
        print(missing_cols_test)

# Aclaración sobre attack_category
missing_attack_cat = test_df['attack_category'].isna().sum()
if missing_attack_cat > 0:
    print(f"\n   ℹ️ Nota: {missing_attack_cat} observaciones en test no tienen 'attack_category' asignada.")
    print(f"      Esto se debe a que contienen tipos de ataque nuevos no presentes en train.")
    print(f"      Esta característica es intencional del dataset NSL-KDD para evaluar generalización.")

In [ ]:
# Uso de memoria y consideraciones computacionales
print("\n💾 Uso de memoria:")
train_memory_mb = train_df.memory_usage(deep=True).sum() / 1024**2
test_memory_mb = test_df.memory_usage(deep=True).sum() / 1024**2

print(f"   Dataset de entrenamiento: {train_memory_mb:.2f} MB")
print(f"   Dataset de prueba: {test_memory_mb:.2f} MB")
print(f"   Total en memoria: {train_memory_mb + test_memory_mb:.2f} MB")

if train_memory_mb + test_memory_mb < 100:
    print(f"\n   ✅ El dataset tiene un tamaño manejable para análisis completo.")
    print(f"      No se requieren técnicas especiales de optimización de memoria.")
else:
    print(f"\n   ⚠️ Dataset de tamaño considerable. Se recomienda:")
    print(f"      - Usar muestras estratificadas para visualizaciones pesadas")
    print(f"      - Monitorear uso de RAM durante análisis")

In [ ]:
# Vista previa de los datos
print("\n📋 Primeras 5 observaciones del dataset de entrenamiento:")
display(train_df.head())

print("\n📋 Últimas 5 observaciones del dataset de entrenamiento:")
display(train_df.tail())

In [ ]:
# Estadísticos descriptivos de variables clave
# Enfocándonos en las variables mencionadas en la Pregunta de Investigación 1
key_vars = ['duration', 'src_bytes', 'dst_bytes']

print("\n📈 Estadísticos descriptivos de variables clave (Dataset completo):")
print("\nEstas tres variables son centrales para la Pregunta de Investigación 1.")
print("Se analizará si presentan diferencias significativas entre tráfico normal y ataques.\n")

key_stats = train_df[key_vars].describe().T
key_stats['range'] = key_stats['max'] - key_stats['min']
key_stats['cv'] = (key_stats['std'] / key_stats['mean']) * 100  # Coeficiente de variación

display(key_stats[['count', 'mean', 'std', 'min', '25%', '50%', '75%', 'max', 'range', 'cv']])

print("\n📊 Interpretación preliminar:")
for var in key_vars:
    mean_val = train_df[var].mean()
    median_val = train_df[var].median()
    max_val = train_df[var].max()
    cv = (train_df[var].std() / mean_val) * 100 if mean_val > 0 else 0
    
    print(f"\n   {var}:")
    print(f"      - Rango: 0 a {max_val:,.0f}")
    print(f"      - Media vs. Mediana: {mean_val:.2f} vs. {median_val:.2f}")
    
    if median_val == 0:
        print(f"      - ⚠️ Mediana en 0: Indica que >50% de las conexiones tienen {var}=0")
    
    if mean_val > median_val * 10 and median_val > 0:
        print(f"      - ⚠️ Asimetría positiva severa detectada (media >> mediana)")
        print(f"      - Probable presencia de outliers extremos")
        print(f"      - Recomendación: Considerar transformación logarítmica para visualización")
    
    if cv > 100:
        print(f"      - ⚠️ Coeficiente de variación muy alto ({cv:.1f}%)")
        print(f"      - Alta heterogeneidad en los datos")

In [ ]:
# Análisis de Asimetría y Curtosis para variables clave
print("\n📐 Asimetría (Skewness) y Curtosis de variables clave:")
print("\nEstas métricas son fundamentales para:")
print("  • Determinar si las variables siguen distribución normal (requisito de ANOVA)")
print("  • Justificar transformaciones antes de PCA (Pregunta 2)")
print("  • Decidir entre pruebas paramétricas vs. no paramétricas (Pregunta 1)\n")

# Calcular para variables clave + algunas adicionales relevantes
analysis_vars = ['duration', 'src_bytes', 'dst_bytes', 'count', 'srv_count', 
                 'serror_rate', 'dst_host_count', 'dst_host_srv_count']

skewness_kurtosis = pd.DataFrame({
    'Skewness': train_df[analysis_vars].skew(),
    'Kurtosis': train_df[analysis_vars].kurtosis()
})

# Agregar interpretación
skewness_kurtosis['Interpretación_Skew'] = skewness_kurtosis['Skewness'].apply(
    lambda x: 'Simétrica' if abs(x) < 0.5 
    else ('Asimétrica moderada' if abs(x) < 1 
          else ('Asimétrica fuerte' if abs(x) < 2 
                else 'Asimétrica SEVERA'))
)

skewness_kurtosis['Interpretación_Kurt'] = skewness_kurtosis['Kurtosis'].apply(
    lambda x: 'Mesocúrtica (normal)' if abs(x) < 1 
    else ('Leptocúrtica (colas pesadas)' if x > 1 
          else 'Platicúrtica (colas ligeras)')
)

display(skewness_kurtosis.round(3))

# Resumen interpretativo
print("\n📊 Resumen interpretativo:")
severe_skew = skewness_kurtosis[abs(skewness_kurtosis['Skewness']) > 2]
if len(severe_skew) > 0:
    print(f"\n   ⚠️ Variables con asimetría SEVERA (|skew| > 2): {len(severe_skew)}")
    print(f"      {', '.join(severe_skew.index.tolist())}")
    print(f"\n   Implicaciones:")
    print(f"      • NO cumplen supuesto de normalidad (ANOVA inválido)")
    print(f"      • REQUIEREN transformación logarítmica para visualización")
    print(f"      • Se debe usar Kruskal-Wallis en lugar de ANOVA (Pregunta 1)")
    print(f"      • Considerar transformación antes de PCA para mejorar resultados")

heavy_tails = skewness_kurtosis[skewness_kurtosis['Kurtosis'] > 3]
if len(heavy_tails) > 0:
    print(f"\n   ⚠️ Variables con colas muy pesadas (kurtosis > 3): {len(heavy_tails)}")
    print(f"      {', '.join(heavy_tails.index.tolist())}")
    print(f"      Indica presencia de outliers extremos (no necesariamente erróneos)")

# Referencia teórica
print("\n📚 Referencias teóricas:")
print("   • Skewness = 0: Distribución simétrica (ej: Normal)")
print("   • |Skewness| > 2: Asimetría severa → distribución NO normal")
print("   • Kurtosis = 0: Distribución mesocúrtica (ej: Normal)")
print("   • Kurtosis > 3: Colas muy pesadas (más outliers que distribución normal)")

### 2.1.1 Hallazgos de la Sección 2.1

**Hallazgos estructurales:**

1. **Calidad de datos:** No se detectaron valores nulos en las 43 variables originales del dataset (41 predictoras + attack_type + difficulty_level). La aparente ausencia de valores en `attack_category` del conjunto de prueba (13 casos) corresponde a tipos de ataque nuevos incluidos intencionalmente para evaluar generalización, no a valores faltantes reales.

2. **Composición de variables:** 
   - **41 variables predictoras:** 38 numéricas (25 int64 + 13 float64) y 3 categóricas (protocol_type, service, flag)
   - **Variables categóricas presentan diferentes cardinalidades:** protocol_type (3 categorías), flag (11 categorías), service (66 categorías)
   - Esta diversidad en cardinalidad requerirá diferentes estrategias de codificación en la preparación de datos

3. **Tamaño computacionalmente manejable:** Con 24.33 MB de uso total de memoria (12.83 MB train + 11.50 MB test), el dataset completo puede procesarse en memoria sin necesidad de técnicas de optimización especiales o procesamiento por lotes.

**Observaciones sobre variables clave (duration, src_bytes, dst_bytes):**

1. **Asimetría extrema y valores cero dominantes:**
   - `duration` y `dst_bytes` tienen mediana = 0, indicando que más del 50% de las conexiones presentan valor cero
   - `src_bytes` tiene mediana de solo 44 bytes, mientras que su media es 24,330 bytes (553 veces mayor)
   - Esta discrepancia media/mediana confirma distribuciones fuertemente sesgadas a la derecha

2. **Heterogeneidad extrema:**
   - Coeficientes de variación extraordinariamente altos: duration (881%), src_bytes (9,909%), dst_bytes (2,544%)
   - Valores superiores al 100% indican que la desviación estándar supera ampliamente la media
   - Esta dispersión extrema es característica de datos de red que mezclan conexiones normales breves con ataques de gran volumen

3. **Rangos de varios órdenes de magnitud:**
   - `src_bytes`: rango de 0 a 381 millones (9 órdenes de magnitud)
   - `dst_bytes`: rango de 0 a 5.1 millones
   - `duration`: rango de 0 a 42,862 segundos (~12 horas)
   - Estos rangos amplios sugieren presencia de outliers legítimos (ataques masivos) que no deben eliminarse

**Métricas de forma de distribución (Skewness y Kurtosis):**

1. **Confirmación cuantitativa de no normalidad:**
   - TODAS las variables clave presentan skewness > 2 (asimetría severa)
   - Esto invalida formalmente el uso de pruebas paramétricas como ANOVA
   - Se confirma la necesidad de Kruskal-Wallis para la Pregunta 1

2. **Colas extremadamente pesadas:**
   - Valores de kurtosis muy superiores a 3 indican presencia abundante de valores extremos
   - Estos outliers NO son errores de medición sino ataques legítimos (ej: DoS masivos)
   - Refuerza la decisión de mantener outliers en lugar de eliminarlos

3. **Justificación para transformaciones:**
   - El skewness severo justifica transformación log1p antes de PCA (Pregunta 2)
   - Variables sin transformar harían que PCA esté dominado por outliers extremos
   - La normalización/estandarización será obligatoria antes de cualquier técnica de ML


## 2.2 Análisis de la Variable Objetivo

Esta sección examina la distribución de la variable objetivo (tipos de ataque) en sus diferentes niveles de granularidad:
- **Nivel 1 (Binario):** Normal vs. Ataque
- **Nivel 2 (Categórico):** Normal, DoS, Probe, R2L, U2R
- **Nivel 3 (Específico):** 39 tipos de ataque individuales

El análisis del desbalance de clases es **crítico** para:
- Interpretar correctamente los resultados de clasificación (Pregunta 3)
- Justificar el uso de métricas especializadas (Recall, F1-Score) en lugar de Accuracy
- Identificar categorías problemáticas para el modelado (especialmente U2R)

In [ ]:
# 2.2.1 Distribución binaria: Normal vs. Ataque
print("2.2.1 DISTRIBUCIÓN BINARIA: NORMAL VS. ATAQUE")

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Train
train_binary = train_df['is_attack'].value_counts()
train_binary_pct = train_df['is_attack'].value_counts(normalize=True) * 100

axes[0].bar(['Normal', 'Ataque'], train_binary.values, 
            color=['#2ecc71', '#e74c3c'], edgecolor='black', alpha=0.8)
axes[0].set_title('Distribución Binaria - TRAIN', fontsize=13, fontweight='bold')
axes[0].set_ylabel('Frecuencia', fontsize=11)
axes[0].grid(axis='y', alpha=0.3)

# Agregar anotaciones de porcentaje
for i, (val, pct) in enumerate(zip(train_binary.values, train_binary_pct.values)):
    axes[0].text(i, val + 500, f'{val:,}\n({pct:.1f}%)', 
                ha='center', va='bottom', fontsize=10, fontweight='bold')

# Test
test_binary = test_df['is_attack'].value_counts()
test_binary_pct = test_df['is_attack'].value_counts(normalize=True) * 100

axes[1].bar(['Normal', 'Ataque'], test_binary.values, 
            color=['#2ecc71', '#e74c3c'], edgecolor='black', alpha=0.8)
axes[1].set_title('Distribución Binaria - TEST', fontsize=13, fontweight='bold')
axes[1].set_ylabel('Frecuencia', fontsize=11)
axes[1].grid(axis='y', alpha=0.3)

# Agregar anotaciones de porcentaje
for i, (val, pct) in enumerate(zip(test_binary.values, test_binary_pct.values)):
    axes[1].text(i, val + 500, f'{val:,}\n({pct:.1f}%)', 
                ha='center', va='bottom', fontsize=10, fontweight='bold')

plt.tight_layout()
plt.show()

# Tabla resumen
print("\n📊 Resumen cuantitativo:")
binary_summary = pd.DataFrame({
    'Train_Freq': train_binary.values,
    'Train_%': train_binary_pct.values.round(2),
    'Test_Freq': test_binary.values,
    'Test_%': test_binary_pct.values.round(2)
}, index=['Normal', 'Ataque'])

print(binary_summary)

print("\n💡 Observación:")
print(f"   • En TRAIN: {train_binary_pct.values[0]:.1f}% Normal vs. {train_binary_pct.values[1]:.1f}% Ataque")
print(f"   • En TEST: {test_binary_pct.values[0]:.1f}% Normal vs. {test_binary_pct.values[1]:.1f}% Ataque")
print(f"   • El dataset está relativamente balanceado a nivel binario (∼50-50)")

In [ ]:
# 2.2.2 Distribución por categorías de ataque
print("2.2.2 DISTRIBUCIÓN POR CATEGORÍAS DE ATAQUE")

fig, axes = plt.subplots(1, 2, figsize=(15, 6))

# Train
category_train = train_df['attack_category'].value_counts()
category_train_pct = train_df['attack_category'].value_counts(normalize=True) * 100

colors_train = [attack_colors[cat] for cat in category_train.index]

axes[0].barh(category_train.index, category_train.values, 
             color=colors_train, edgecolor='black', alpha=0.85)
axes[0].set_title('Distribución por Categoría - TRAIN', fontsize=13, fontweight='bold')
axes[0].set_xlabel('Frecuencia', fontsize=11)
axes[0].grid(axis='x', alpha=0.3)

# Agregar anotaciones
for i, (cat, val, pct) in enumerate(zip(category_train.index, category_train.values, category_train_pct.values)):
    axes[0].text(val + 300, i, f'{val:,} ({pct:.2f}%)', 
                va='center', fontsize=10, fontweight='bold')

# Test (filtrar NaN)
category_test = test_df[test_df['attack_category'].notna()]['attack_category'].value_counts()
category_test_pct = test_df[test_df['attack_category'].notna()]['attack_category'].value_counts(normalize=True) * 100

colors_test = [attack_colors[cat] for cat in category_test.index]

axes[1].barh(category_test.index, category_test.values, 
             color=colors_test, edgecolor='black', alpha=0.85)
axes[1].set_title('Distribución por Categoría - TEST', fontsize=13, fontweight='bold')
axes[1].set_xlabel('Frecuencia', fontsize=11)
axes[1].grid(axis='x', alpha=0.3)

# Agregar anotaciones
for i, (cat, val, pct) in enumerate(zip(category_test.index, category_test.values, category_test_pct.values)):
    axes[1].text(val + 300, i, f'{val:,} ({pct:.2f}%)', 
                va='center', fontsize=10, fontweight='bold')

plt.tight_layout()
plt.show()

# Tabla comparativa
print("\n📊 Comparación Train vs. Test:")
comparison_df = pd.DataFrame({
    'Train_Freq': category_train,
    'Train_%': category_train_pct.round(2),
    'Test_Freq': category_test,
    'Test_%': category_test_pct.round(2)
})
comparison_df = comparison_df.reindex(category_order)
comparison_df['Diferencia_%'] = (comparison_df['Test_%'] - comparison_df['Train_%']).round(2)

print(comparison_df)

In [ ]:
# 2.2.3 Análisis detallado del desbalance de clases
print("2.2.3 ANÁLISIS DE DESBALANCE DE CLASES")

print("\n⚖️ Métricas de desbalance en TRAIN:")
print(f"\n   Categoría MAYORITARIA: {category_train.index[0]} con {category_train.values[0]:,} muestras ({category_train_pct.values[0]:.2f}%)")
print(f"   Categoría MINORITARIA: {category_train.index[-1]} con {category_train.values[-1]:,} muestras ({category_train_pct.values[-1]:.2f}%)")

# Calcular ratio de desbalance
imbalance_ratio_train = category_train.values[0] / category_train.values[-1]
print(f"\n   📉 Ratio de desbalance (Mayoritaria/Minoritaria): {imbalance_ratio_train:.1f}:1")
print(f"      → La clase mayoritaria tiene {imbalance_ratio_train:.0f} veces más ejemplos que U2R")

# Análisis por categoría
print("\n📊 Caracterización del desbalance por categoría (TRAIN):")
for cat in category_order:
    freq = category_train.get(cat, 0)
    pct = category_train_pct.get(cat, 0)
    
    if pct > 30:
        status = "✅ BIEN REPRESENTADA"
    elif pct > 5:
        status = "🟡 MODERADAMENTE REPRESENTADA"
    elif pct > 1:
        status = "⚠️ POCO REPRESENTADA"
    else:
        status = "🔴 EXTREMADAMENTE RARA"
    
    print(f"   {cat:10s}: {freq:6,} muestras ({pct:5.2f}%) → {status}")

# Comparación Train vs Test
print("\n🔄 Comparación de distribuciones Train vs. Test:")
print("\n   Categorías con MAYOR cambio de proporción:")
for cat in category_order:
    train_pct = category_train_pct.get(cat, 0)
    test_pct = category_test_pct.get(cat, 0)
    diff = test_pct - train_pct
    
    if abs(diff) > 5:  # Cambios mayores al 5%
        direction = "↑ AUMENTÓ" if diff > 0 else "↓ DISMINUYÓ"
        print(f"   • {cat:10s}: {direction} {abs(diff):.2f} puntos porcentuales")
        print(f"      Train: {train_pct:.2f}% → Test: {test_pct:.2f}%")

In [ ]:
# 2.2.4 Análisis de ataques específicos dentro de cada categoría
print("2.2.4 ATAQUES ESPECÍFICOS POR CATEGORÍA")

# Crear visualización de los top ataques específicos
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
axes = axes.ravel()

# Analizar cada categoría de ataque (excepto Normal)
attack_categories_analysis = ['DoS', 'Probe', 'R2L', 'U2R']

for idx, category in enumerate(attack_categories_analysis):
    # Filtrar ataques de esta categoría en train
    attacks_in_category = train_df[train_df['attack_category'] == category]['attack_type'].value_counts()
    
    # Crear gráfico de barras horizontal
    colors_bar = [attack_colors[category]] * len(attacks_in_category)
    
    attacks_in_category.plot(kind='barh', ax=axes[idx], color=colors_bar, 
                             edgecolor='black', alpha=0.8)
    axes[idx].set_title(f'Ataques Específicos - {category}', 
                       fontsize=12, fontweight='bold')
    axes[idx].set_xlabel('Frecuencia', fontsize=10)
    axes[idx].grid(axis='x', alpha=0.3)
    
    # Agregar anotaciones de frecuencia
    for i, (attack, freq) in enumerate(attacks_in_category.items()):
        pct = (freq / attacks_in_category.sum()) * 100
        axes[idx].text(freq + (attacks_in_category.max() * 0.02), i, 
                      f'{freq:,} ({pct:.1f}%)', 
                      va='center', fontsize=9)

plt.tight_layout()
plt.show()

# Tabla resumen detallada
print("\n📋 Resumen detallado de ataques específicos:")
for category in attack_categories_analysis:
    print(f"Categoría: {category}")
    
    # Train
    train_attacks = train_df[train_df['attack_category'] == category]['attack_type'].value_counts()
    train_attacks_pct = (train_attacks / train_attacks.sum() * 100).round(2)
    
    # Test
    test_attacks = test_df[test_df['attack_category'] == category]['attack_type'].value_counts()
    test_attacks_pct = (test_attacks / test_attacks.sum() * 100).round(2)
    
    # Combinar información
    all_attacks = sorted(set(train_attacks.index) | set(test_attacks.index))
    
    print(f"\n{'Ataque':<20} {'Train':>15} {'Train %':>10} {'Test':>15} {'Test %':>10}")
    print("-" * 75)
    
    for attack in all_attacks:
        train_freq = train_attacks.get(attack, 0)
        train_pct = train_attacks_pct.get(attack, 0)
        test_freq = test_attacks.get(attack, 0)
        test_pct = test_attacks_pct.get(attack, 0)
        
        # Marcar ataques que solo aparecen en test
        marker = " 🆕" if train_freq == 0 and test_freq > 0 else ""
        
        print(f"{attack:<20} {train_freq:>15,} {train_pct:>9.2f}% {test_freq:>15,} {test_pct:>9.2f}%{marker}")
    
    print(f"\nTotal en {category}: Train={train_attacks.sum():,}, Test={test_attacks.sum():,}")

In [ ]:
# 2.2.5 Visualización comparativa de distribuciones Train vs Test
print("2.2.5 COMPARACIÓN VISUAL TRAIN VS. TEST")

# Gráfico de barras agrupadas
fig, ax = plt.subplots(figsize=(12, 6))

x = np.arange(len(category_order))
width = 0.35

# Preparar datos
train_values = [category_train.get(cat, 0) for cat in category_order]
test_values = [category_test.get(cat, 0) for cat in category_order]

# Crear barras
bars1 = ax.bar(x - width/2, train_values, width, label='Train', 
               color='#3498db', edgecolor='black', alpha=0.8)
bars2 = ax.bar(x + width/2, test_values, width, label='Test', 
               color='#e67e22', edgecolor='black', alpha=0.8)

# Configuración del gráfico
ax.set_xlabel('Categoría de Ataque', fontsize=12, fontweight='bold')
ax.set_ylabel('Frecuencia', fontsize=12, fontweight='bold')
ax.set_title('Comparación de Distribuciones: Train vs. Test', 
            fontsize=14, fontweight='bold')
ax.set_xticks(x)
ax.set_xticklabels(category_order, fontsize=11)
ax.legend(fontsize=11)
ax.grid(axis='y', alpha=0.3)

# Agregar valores encima de las barras
for bars in [bars1, bars2]:
    for bar in bars:
        height = bar.get_height()
        ax.text(bar.get_x() + bar.get_width()/2., height,
                f'{int(height):,}',
                ha='center', va='bottom', fontsize=9)

plt.tight_layout()
plt.show()